# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2a24cc90d0>
├── 'a' --> tensor([[-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580]])
└── 'x' --> <FastTreeValue 0x7f2a24c99b20>
    └── 'c' --> tensor([[ 0.1077, -0.7732, -0.0566,  1.7932],
                        [-0.2822, -1.9354, -0.1867,  1.8209],
                        [ 0.1375, -0.1942, -0.2934, -0.4540]])

In [4]:
t.a

tensor([[-1.0707,  1.2092,  0.9996],
        [ 0.6008,  0.5328,  1.0580]])

In [5]:
%timeit t.a

78.3 ns ± 0.232 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2a24cc90d0>
├── 'a' --> tensor([[-0.3461,  0.0019,  0.5644],
│                   [-0.7596,  0.4651, -1.8409]])
└── 'x' --> <FastTreeValue 0x7f2a24c99b20>
    └── 'c' --> tensor([[ 0.1077, -0.7732, -0.0566,  1.7932],
                        [-0.2822, -1.9354, -0.1867,  1.8209],
                        [ 0.1375, -0.1942, -0.2934, -0.4540]])

In [7]:
%timeit t.a = new_value

90.5 ns ± 0.243 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0707,  1.2092,  0.9996],
               [ 0.6008,  0.5328,  1.0580]]),
    x: Batch(
           c: tensor([[ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540]]),
       ),
)

In [10]:
b.a

tensor([[-1.0707,  1.2092,  0.9996],
        [ 0.6008,  0.5328,  1.0580]])

In [11]:
%timeit b.a

69.1 ns ± 0.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0648,  0.5701,  0.3449],
               [-0.6834,  0.1541, -0.1849]]),
    x: Batch(
           c: tensor([[ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540]]),
       ),
)

In [13]:
%timeit b.a = new_value

599 ns ± 2.75 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

977 ns ± 6.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 79.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 966 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 557 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f29fc97b2e0>
├── 'a' --> tensor([[[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]],
│           
│                   [[-1.0707,  1.2092,  0.9996],
│                    [ 0.6008,  0.5328,  1.0580]]])
└── 'x' --> <FastTreeValue 0x7f29fc97b310>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37 µs ± 176 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2a24cc9160>
├── 'a' --> tensor([[-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580],
│                   [-1.0707,  1.2092,  0.9996],
│                   [ 0.6008,  0.5328,  1.0580]])
└── 'x' --> <FastTreeValue 0x7f2a24c94310>
    └── 'c' --> tensor([[ 0.1077, -0.7732, -0.0566,  1.7932],
                        [-0.2822, -1.9354, -0.1867,  1.8209],
                 

In [23]:
%timeit t_cat(trees)

34.2 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.9 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1077, -0.7732, -0.0566,  1.7932],
                       [-0.2822, -1.9354, -0.1867,  1.8209],
                       [ 0.1375, -0.1942, -0.2934, -0.4540]],
              
                      [[ 0.1077, -0.7732, -0.0566,  1.7932],
                       [-0.2822, -1.9354, -0.1867,  1.8209],
                       [ 0.1375, -0.1942, -0.2934, -0.4540]],
              
                      [[ 0.1077, -0.7732, -0.0566,  1.7932],
                       [-0.2822, -1.9354, -0.1867,  1.8209],
                       [ 0.1375, -0.1942, -0.2934, -0.4540]],
              
                      [[ 0.1077, -0.7732, -0.0566,  1.7932],
                       [-0.2822, -1.9354, -0.1867,  1.8209],
                       [ 0.1375, -0.1942, -0.2934, -0.4540]],
              
                      [[ 0.1077, -0.7732, -0.0566,  1.7932],
                       [-0.2822, -1.9354, -0.1867,  1.8209],
                       [ 0.1375, -0.1942, -0.2934, -0.4540]],

In [26]:
%timeit Batch.stack(batches)

89.2 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540],
                      [ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540],
                      [ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540],
                      [ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540],
                      [ 0.1077, -0.7732, -0.0566,  1.7932],
                      [-0.2822, -1.9354, -0.1867,  1.8209],
                      [ 0.1375, -0.1942, -0.2934, -0.4540],
                      [ 0.1077, -0.7732, -0.0566,  1.7932],
                   

In [28]:
%timeit Batch.cat(batches)

166 µs ± 691 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

380 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
